In [1]:
# importing Required Libraries
import numpy as np
import pandas as pd
import ast
import warnings
warnings.filterwarnings("ignore")
from bs4 import BeautifulSoup
import requests
import re
# pd.set_option('max_colwidth', None)

In [2]:
#loading data
internal_data = pd.read_csv("internal_file_data.csv")

In [3]:
internal_data.head()

,Unnamed: 0,name,Wealth_History,personal_data
0,0,Elon Musk,"[{""percent"":3.8356164383561646,""worth"":""$8.4B""...","Age52Source of WealthTesla, SpaceX, Self MadeS..."
1,1,Bernard Arnault & family,"[{""percent"":15.876777251184834,""worth"":""$33.5B...","Age74Source of WealthLVMHResidenceParis, Franc..."
2,2,Jeff Bezos,"[{""percent"":18.07909604519774,""worth"":""$32B"",""...","Age59Source of WealthAmazon, Self MadeSelf-Mad..."
3,3,Larry Ellison,"[{""percent"":44.85981308411215,""worth"":""$48B"",""...","Age78Source of WealthOracle, Self MadeSelf-Mad..."
4,4,Warren Buffett,"[{""percent"":49.32203389830509,""worth"":""$58.2B""...","Age92Source of WealthBerkshire Hathaway, Self ..."


In [4]:
internal_data.shape

(2609, 4)

In [5]:
internal_data.isnull().sum()

Unnamed: 0          0
name                4
Wealth_History    229
personal_data       4
dtype: int64

In [6]:
internal_data[internal_data['name'].isnull()]

,Unnamed: 0,name,Wealth_History,personal_data
41,41,NaN,NaN,NaN
95,95,NaN,NaN,NaN
2021,2021,NaN,NaN,NaN
2498,2498,NaN,NaN,NaN


## Summary for the Data

This is a Dataset containing information of billionaires from all over the world with more than 2600 rows mentioned on **realtime forbes billinaires list of 2023**. This dataset has three columns, 'Name' of the billionaires, their wealth history of last 6 to 8 years, and their personal information like citizenship, married or something else, and their Education in "personal_data" column.


### Column's Description

- `Name`: Name of the billionaires.

- `Wealth_History`: This column holds the information of billionaires's worth value(in billion dollar) in a particular year. Ex - ([Elon Musk] - (2014, 8.4B),(2015,12B),........

- `personal_data`: This column has many information about billionaires. What and Where did they study?, Where do they live? The company was made by him/her or inherited. Etc..

#### As this data is too messey and dirty, Further I will be doing data cleaning.

#### Issues With the Dataset

- `Unnamed: 0` - This column is of no use.

- `Name` - In name column, some rows contains **& family** with their name in last and some nan values are also there. - `Dirty Data`

- `Wealth_History` - This column contains wealth value of particular year between 2011 to 2023 in the form of dictionary and some nan values are also there. - `Messy Data` 

- `personal_data` - This column has many information, like Education, Citizenship etc, all in one column. - `Messy Data`

## Data Cleaning - Part 1

- Remove **Unnamed: 0** column
- Remove **& family** from name column, 4 values are nan in name column and also there are nan in corresponding column, So drop nan values.
- Extract wealth value with corresponding year value in tuple from wealth_history column. Do not drop nan values.
- Personal_data column has many information, Extract **Citizenship, Marital Status, Education** and create is coulumn **Empire** that will tell, Business was self made or something else.
- Some of the billionaires educations are multiple, Extract first Education and its College/Univrsity.

### Remove Unnamed: 0 column

In [7]:
internal_data.drop(columns = 'Unnamed: 0', inplace=True)

### Remove & family from name column, 3 values are nan in name column and also there are nan in corresponding column, So drop nan values.

In [8]:
internal_data[internal_data['name'].str.contains('& family', na=False)]

,name,Wealth_History,personal_data
1,Bernard Arnault & family,"[{""percent"":15.876777251184834,""worth"":""$33.5B...","Age74Source of WealthLVMHResidenceParis, Franc..."
11,Carlos Slim Helu & family,"[{""percent"":77.41935483870968,""worth"":""$72B"",""...","Age83Source of WealthTelecom, Self MadeResiden..."
13,Francoise Bettencourt Meyers & family,"[{""percent"":52.422360248447205,""worth"":""$42.2B...","Age70Source of WealthL'OréalResidenceParis, Fr..."
20,Julia Koch & family,"[{""percent"":63.66666666666667,""worth"":""$38.2B""...",Age61Source of WealthKoch IndustriesSelf-Made ...
22,David Thomson & family,"[{""percent"":41.544117647058826,""worth"":""$22.6B...","Age66Source of WealthMediaResidenceToronto, Ca..."
...,...,...,...
2548,Zhen Li & family,"[{""percent"":68.75,""worth"":""$1.1B"",""name"":""Bill...","Age59Source of WealthLithium batteries, Self M..."
2550,David Tran & family,NaN,"Age77Source of WealthHot sauce, Self MadeResid..."
2570,Huang Xiaofen & family,"[{""percent"":66.66666666666666,""worth"":""$1B"",""n...","Age61Source of WealthPrinted circuit boards, S..."
2592,Du Yulin & family,NaN,"Age51Source of WealthSoftware, Self MadeReside..."


In [9]:
internal_data['name'] = internal_data['name'].str.replace('& family','').str.strip()

In [10]:
internal_data[internal_data['name'].isnull()]
# Will drop three based on index value

,name,Wealth_History,personal_data
41,NaN,NaN,NaN
95,NaN,NaN,NaN
2021,NaN,NaN,NaN
2498,NaN,NaN,NaN


In [11]:
internal_data.drop([41, 95, 2021, 2498], inplace=True)

In [12]:
internal_data

,name,Wealth_History,personal_data
0,Elon Musk,"[{""percent"":3.8356164383561646,""worth"":""$8.4B""...","Age52Source of WealthTesla, SpaceX, Self MadeS..."
1,Bernard Arnault,"[{""percent"":15.876777251184834,""worth"":""$33.5B...","Age74Source of WealthLVMHResidenceParis, Franc..."
2,Jeff Bezos,"[{""percent"":18.07909604519774,""worth"":""$32B"",""...","Age59Source of WealthAmazon, Self MadeSelf-Mad..."
3,Larry Ellison,"[{""percent"":44.85981308411215,""worth"":""$48B"",""...","Age78Source of WealthOracle, Self MadeSelf-Mad..."
4,Warren Buffett,"[{""percent"":49.32203389830509,""worth"":""$58.2B""...","Age92Source of WealthBerkshire Hathaway, Self ..."
...,...,...,...
2604,Zhong Peifeng,"[{""percent"":64.70588235294117,""worth"":""$1.1B"",...","Age60Source of Wealthsemiconductor, Self MadeR..."
2605,Kanye West,"[{""percent"":90,""worth"":""$1.8B"",""name"":""Billion...","Age46Source of Wealthmusic, sneakers, Self Mad..."
2606,Chung Yong-ji,NaN,"Age53Source of WealthBiotech, Self MadeResiden..."
2607,Sam Bankman-Fried,"[{""percent"":36.25,""worth"":""$8.7B"",""name"":""Bill...","Age31Source of Wealthcryptocurrency exchange, ..."


### Extract wealth value with corresponding year value in tuple from wealth_history column.

In [13]:
# Removing [ , ] brackets so that I could get dictionaries inside it.
internal_data['Wealth_History'] = internal_data['Wealth_History'].str.replace("[",'').str.replace("]",'')

In [14]:
internal_data['Wealth_History']

0       {"percent":3.8356164383561646,"worth":"$8.4B",...
1       {"percent":15.876777251184834,"worth":"$33.5B"...
2       {"percent":18.07909604519774,"worth":"$32B","n...
3       {"percent":44.85981308411215,"worth":"$48B","n...
4       {"percent":49.32203389830509,"worth":"$58.2B",...
                              ...                        
2604    {"percent":64.70588235294117,"worth":"$1.1B","...
2605    {"percent":90,"worth":"$1.8B","name":"Billiona...
2606                                                  NaN
2607    {"percent":36.25,"worth":"$8.7B","name":"Billi...
2608    {"percent":100,"worth":"$4.5B","name":"Billion...
Name: Wealth_History, Length: 2605, dtype: object

In [15]:
# Extracting wealth value with corresponding year in tuple.
wealth = []
for i in internal_data['Wealth_History']:
    individuaal_wealth = []
    if pd.isna(i):
        wealth.append(np.nan)
    else:
        x = ast.literal_eval(i)
        for j in x:
            worth = j['worth'].replace('$','')
            date = j['date'].split(' ')[1]
            value = (date, worth)
            individuaal_wealth.append(value)
        wealth.append(individuaal_wealth)     

In [16]:
internal_data['Wealth_History'] = wealth

### Extract Citizenship from personal_data column.

In [17]:
internal_data['Citizenship'] = internal_data['personal_data'].str.split('Citizenship').str.get(1).str.split('Marital').str.get(0)
internal_data['Citizenship'] = internal_data['Citizenship'].str.split('Education').str.get(0)
internal_data['Citizenship'] = internal_data['Citizenship'].str.split('Child').str.get(0)

### Extract Marital Status from personal_data column.

In [18]:
internal_data['Marital Status'] = internal_data['personal_data'].str.split('Marital Status').str.get(1).str.split('Children').str.get(0)
internal_data['Marital Status'] = internal_data['Marital Status'].str.split('Education').str.get(0)

### Extract First Education from personal_data column.

In [19]:
internal_data['Education'] = internal_data['personal_data'].str.split('Education').str.get(1).str.split(',').str.get(0)

### Extract First College/University from personal_data column.

In [20]:
internal_data['College/University'] = internal_data['personal_data'].str.split('Education').str.get(1).str.split(';').str.get(0)

In [21]:
internal_data['College/University'] = internal_data['College/University'].str.split(',').str.get(-1).str.strip()

### Extract 'Self-made' value from personal_data column.

In [22]:
Empire = internal_data['personal_data'].str.split('Residence').str.get(0)

In [23]:
empire = []
for i in Empire:
    if 'Self Made' in i:
        empire.append("Self Made")
    else:
        empire.append(np.nan)

In [24]:
internal_data['Empire'] = empire

In [25]:
internal_data

,name,Wealth_History,personal_data,Citizenship,Marital Status,Education,College/University,Empire
0,Elon Musk,"[(2014, 8.4B), (2015, 12B), (2016, 10.7B), (20...","Age52Source of WealthTesla, SpaceX, Self MadeS...",United States,Single,Bachelor of Arts/Science,University of Pennsylvania,Self Made
1,Bernard Arnault,"[(2014, 33.5B), (2015, 37.2B), (2016, 34B), (2...","Age74Source of WealthLVMHResidenceParis, Franc...",France,Married,Bachelor of Arts/Science,Ecole Polytechnique de Paris,NaN
2,Jeff Bezos,"[(2014, 32B), (2015, 34.8B), (2016, 45.2B), (2...","Age59Source of WealthAmazon, Self MadeSelf-Mad...",United States,Engaged,Bachelor of Arts/Science,Princeton University,Self Made
3,Larry Ellison,"[(2014, 48B), (2015, 54.3B), (2016, 43.6B), (2...","Age78Source of WealthOracle, Self MadeSelf-Mad...",United States,In Relationship,Drop Out,University of Chicago,Self Made
4,Warren Buffett,"[(2014, 58.2B), (2015, 72.7B), (2016, 60.8B), ...","Age92Source of WealthBerkshire Hathaway, Self ...",United States,"Widowed, Remarried",Master of Science,Columbia University,Self Made
...,...,...,...,...,...,...,...,...
2604,Zhong Peifeng,"[(2021, 1.1B), (2022, 1.7B)]","Age60Source of Wealthsemiconductor, Self MadeR...",United States,Married,NaN,NaN,Self Made
2605,Kanye West,"[(2021, 1.8B), (2022, 2B)]","Age46Source of Wealthmusic, sneakers, Self Mad...",United States,Divorced,Drop Out,Chicago State University,Self Made
2606,Chung Yong-ji,NaN,"Age53Source of WealthBiotech, Self MadeResiden...",South Korea,NaN,Doctorate,Cornell University,Self Made
2607,Sam Bankman-Fried,"[(2021, 8.7B), (2022, 24B)]","Age31Source of Wealthcryptocurrency exchange, ...",United States,NaN,Bachelor of Arts/Science,Massachusetts Institute of Technology,Self Made


##### Now I have extracted all the required information from **Wealth History** and **Personal Data** Column. But There are still many issues in new extracted column. 
In this Data Cleaning - Part 2, I will be cleaning messy and dirty data from these columns.

### Isuues with this Dataset.

- Column `Wealth_History`, `Citizenship`, `Education`, `College/University` have some messy or dirty data.

## Data Cleaning - Part 2

- `CitizenShip Column`
    1. Index **[697]** has **United StatesAgentDavid FalkAgencyFAME** --- Replace with **United States**
    
    2. Index **[2495]** has **United StatesAgentMark SteinbergAgencyExcel Sports ManagementSponsors2K Sports, Autograph, Bridgestone, Full Swing, Kowa Company Ltd., Monster Energy, Rolex, TaylorMade, Upper Deck** --- Replace with **United States**


- `Education Column`   
    1. Index **[1939]** has **Xiamen University** --- Replace with **NaN**.
    2. Index **[964]** has **Empty** --- Replace with **NaN**.
    3. Index **[1720]** has **Empty** --- Replace with **Bachelor of Arts/Science**.
    4. Index **[1958]** has **Empty** --- Replace with **EMBA**.
    5. Index **[2245]** has **Empty** --- Replace with **NaN**.
    6. Index **[2420]** has **Empty** --- Replace with **NaN**.
    7. Index **[355]** has **Houston Community College** --- Replace with **NaN**.
    8. Index **[594]** has **Chinese Academy of Social Sciences; Bachelor of Arts/Science** --- Replace with **Bachelor of Arts/Science**.
    9. Index **[757]** has **Chinese Culture University** --- Replace with **NaN**.
    10. Index **[1036]** has **China Europe International Business School** --- Replace with **NaN**.
    11. Index **[1971]** has **London School of Economics; Master** --- Replace with **NaN**.
    12. Index **[2324]** has **University of Pennsylvania** --- Replace with **NaN**.
    13. Index **[2315]** has **Waseda University** --- Replace with **NaN**.
    14. Index **[1390]** has **Harvard Business School; Bachelor of Arts/Science** --- Replace with **NaN**.
    15. Index **[2291]** has **Oregon State University; Walla Walla College** --- Replace with **NaN**.
    16. Index **[383]** has **National Taipei University of Technology** --- Replace with **NaN**.
    17. Index **[2349]** has **University of California** --- Replace with **NaN**.
    18. Index **[283]** has **State Central Institute of Physical Culture** --- Replace with **NaN**.
    19. Index **[280]** has **Liaoning University** --- Replace with **NaN**.
    20. Index **[1229]** has **Fordham University; Bachelor of Arts/Science** --- Replace with **NaN**.
    21. Index **[1409]** has **City University of New York Baruch** --- Replace with **NaN**.
    22. Index **[1980]** has **Chinese University of Hong Kong** --- Replace with **NaN**.
    23. Index **[909]** has **ResidenceDubai** --- Replace with **NaN**.
    24. Index **[347]** has **Taipei University of Marine Technology** --- Replace with **NaN**.
    25. Index **[1131]** has **ResidenceBangalore** --- Replace with **NaN**.
    26. Index **[1862]** has **ResidencePhoenix** --- Replace with **Bachelor of Arts/Science**.
    27. Index **[1679]** has **Jiangsu Province Wujin Teacher School** --- Replace with **NaN**.
    28. Index **[1620]** has **National Taitung University** --- Replace with **NaN**.
    29. Index **[2214]** has **Sweet Briar College** --- Replace with **NaN**.
    30. Index **[2215]** has **The University of North Carolina at Chapel Hill** --- Replace with **NaN**.
    31. Index **[2495]** has **Stanford University** --- Replace with **NaN**
    

    
    
- `College/University Column` 

    1. Index **[1222]** has **India** --- Replace with **Anna University**.
    2. Index **[204, 251, 679, 705, 728, 764, 1327, 2349, 61, 410, 663, 1666, 2182, 2300, 2322, 2340, 2587]** has **Los Angeles or Berkeley** --- Replace with **University of California**.
    3. Index **[1596]** has **Davis** --- Replace with **University of California**.
    4. Index **[194]** has **Austin** --- Replace with **University of Texas**.
    5. Index **[2024]** has **Kazan** --- Replace with **Moscow Power Engineering Institute**.
    6. Index **[81]** has **Eau Claire** --- Replace with **University of Wisconsin**.
    7. Index **[1082, 1253, 1396]** has **Ahmedabad** --- Replace with **Indian Institute of Management**.
    8. Index **[610]** has **Moscow)** --- Replace with **State Institute of Theatre Arts**.
    9. Index **[1361]** has **Electronics and Automation** --- Replace with **Moscow Institute of Radioengineering**.
    10. Index **[1845]** has **Madison** --- Replace with **University of Wisconsin**.
    11. Index **[1931]** has **marketing/management** --- Replace with **Stockholm University**.
    12. Index **[270]** has **Prague** --- Replace with **Czech Technical University**.
    13. Index **[1527]** has **Chapel Hill** --- Replace with **University of North Carolina**.
    14. Index **[197]** has **San Diego** --- Replace with **University of California**.
    15. Index **[269]** has **BHU** --- Replace with **Indian Institute of Technology**.
    16. Index **[1647]** has **Germany** --- Replace with **Rheinisch-Westfälische Technische Hochschule**.
    17. Index **[1813]** has **The Netherlands** --- Replace with **Erasmus University Rotterdam**.
    18. Index **[1962]** has **Bombay** --- Replace with **Indian Institute of Technology**.
    19. Index **[2050]** has **Jerusalem** --- Replace with **Hebrew University**.
    20. Index **[1129, 1140]** has **Lebanon** --- Replace with **American University of Beirut**.
    21. Index **[143]** has **Ceibs** --- Replace with **NaN**
    22. Index **[372]** has **London** --- Replace with **Sir John Cass College**
    23. Index **[331]** has **Sydney** --- Replace with **University of Technology**
    24. Index **[483]** has **Cambridge** --- Replace with **University College London**
    25. Index **[1862, 2542, 2034]** has **Wrong Value** --- Replace with **University of California**
    26. Index **[2044]** has **Wrong Value** --- Replace with **Europe International Business School**
    27. Index **[2092]** has **India** --- Replace with **Indian Institute of Technology (IIT) Kharagpur**

Some values are dirty(should not be there) those dirty data are mentioned above. Along with that Some of the Colleges/Universities name are not written in English Language rather they are written in their native lanuguage. So I am leaving those names as it is.
   
   
- `Wealth_History Column`   

Create Seperate Column by Year name that will have the worth value if available in wealth history column else NaN


#### Drop Wealth_History and Personal_data column.

- After Creating seperate column from 2014 to 2023, all worth values are in billions, but index **[2563]'s** **2015**, and **2016** values are in million. So, convert it billion. Then **Replace 'B',''** and covert astype to **float**

### Cleaning Citizenship Column

In [26]:
internal_data['Citizenship'].unique()

array(['United States', 'France', 'Mexico', 'India', 'Spain', 'China',
       'Canada', 'Germany', 'Austria', 'Italy', 'Hong Kong', 'Japan',
       'Switzerland', 'Australia', 'Indonesia', 'Russia', 'Chile',
       'United Kingdom', 'Israel', 'Sweden', 'Brazil', 'Czech Republic',
       'Thailand', 'Netherlands', 'Singapore', 'Cyprus',
       'United Arab Emirates', 'Nigeria', 'Malaysia', 'Taiwan',
       'South Africa', 'Philippines', 'New Zealand', 'South Korea',
       'Belgium', 'Colombia', 'Norway', 'Monaco', 'Ireland', 'Egypt',
       'Denmark', 'Poland', 'Ukraine', 'Eswatini (Swaziland)', 'Greece',
       'Vietnam', 'Argentina', 'Turkey', 'Portugal', 'Georgia',
       'Kazakhstan', 'Algeria', 'Venezuela',
       'United StatesAgentDavid FalkAgencyFAME', 'Finland', 'Lebanon',
       'Belize', 'Iceland', 'Oman', 'Liechtenstein', 'Guernsey',
       'Bulgaria', 'Romania', 'Nepal', 'Zimbabwe', 'Qatar', 'Estonia',
       'Slovakia', 'Peru', 'Hungary', 'Morocco', 'Tanzania', 'Barbados'

In [27]:
internal_data.loc[[697, 2495], 'Citizenship'] = 'United States'

In [28]:
internal_data['Citizenship'].unique()

array(['United States', 'France', 'Mexico', 'India', 'Spain', 'China',
       'Canada', 'Germany', 'Austria', 'Italy', 'Hong Kong', 'Japan',
       'Switzerland', 'Australia', 'Indonesia', 'Russia', 'Chile',
       'United Kingdom', 'Israel', 'Sweden', 'Brazil', 'Czech Republic',
       'Thailand', 'Netherlands', 'Singapore', 'Cyprus',
       'United Arab Emirates', 'Nigeria', 'Malaysia', 'Taiwan',
       'South Africa', 'Philippines', 'New Zealand', 'South Korea',
       'Belgium', 'Colombia', 'Norway', 'Monaco', 'Ireland', 'Egypt',
       'Denmark', 'Poland', 'Ukraine', 'Eswatini (Swaziland)', 'Greece',
       'Vietnam', 'Argentina', 'Turkey', 'Portugal', 'Georgia',
       'Kazakhstan', 'Algeria', 'Venezuela', 'Finland', 'Lebanon',
       'Belize', 'Iceland', 'Oman', 'Liechtenstein', 'Guernsey',
       'Bulgaria', 'Romania', 'Nepal', 'Zimbabwe', 'Qatar', 'Estonia',
       'Slovakia', 'Peru', 'Hungary', 'Morocco', 'Tanzania', 'Barbados',
       'St. Kitts and Nevis', 'Armenia', 'Macau

### Cleaning Education Column

In [29]:
internal_data['Education'].unique()

array(['Bachelor of Arts/Science', 'Drop Out', 'Master of Science',
       'Master of Business Administration', nan,
       'Doctor of Jurisprudence', 'Master of Arts',
       'Bachelor of Engineering', 'Bachelor of Science in Engineering',
       'Diploma', 'Bachelor of Science', 'Master',
       'Associate in Arts/Science', 'Masters in Finance', 'Doctorate',
       'Master of Science in Engineering', 'Bachelor of Technology',
       'EMBA', 'High School Graduate', 'Ph.D', 'Master of Economics',
       'Bachelor of Arts/Economics', 'Medical Doctor', 'Graduate', 'LLB',
       'Law Degree', 'Registered Nurse', 'Master of Laws',
       'Liaoning University',
       'State Central Institute of Physical Culture',
       'Master of Arts/Science', 'Bachelor of Business Administration',
       'Taipei University of Marine Technology',
       'Houston Community College',
       'National Taipei University of Technology', 'Bachelor of Commerce',
       'Masters of Public Health',
       'Chines

In [30]:
internal_data.loc[[1939, 964, 2245, 2420, 355, 757, 1036, 1971, 2324, 2315, 1390, 2291, 383, 2349, 283, 280, 1229, 1409, 1980, 909, 347, 1131, 1679, 1620,2214, 2215, 2495], 
'Education'] = np.nan

In [31]:
internal_data.loc[[1720, 594, 1862, 2598], 'Education'] = 'Bachelor of Arts/Science'

In [32]:
internal_data.loc[1958, 'Education'] = 'EMBA'

In [33]:
internal_data['Education'].unique()

array(['Bachelor of Arts/Science', 'Drop Out', 'Master of Science',
       'Master of Business Administration', nan,
       'Doctor of Jurisprudence', 'Master of Arts',
       'Bachelor of Engineering', 'Bachelor of Science in Engineering',
       'Diploma', 'Bachelor of Science', 'Master',
       'Associate in Arts/Science', 'Masters in Finance', 'Doctorate',
       'Master of Science in Engineering', 'Bachelor of Technology',
       'EMBA', 'High School Graduate', 'Ph.D', 'Master of Economics',
       'Bachelor of Arts/Economics', 'Medical Doctor', 'Graduate', 'LLB',
       'Law Degree', 'Registered Nurse', 'Master of Laws',
       'Master of Arts/Science', 'Bachelor of Business Administration',
       'Bachelor of Commerce', 'Masters of Public Health',
       'Doctorate in Pedagogical Sciences', 'LLM',
       'Bachelor of Social Sciences', 'Master of Philosophy',
       'Master of Engineering Management', 'B.Com',
       'The Wharton School at the University of Pennsylvania',
      

### Cleaning College/University Column

In [34]:
internal_data['College/University'].unique()

array(['University of Pennsylvania', 'Ecole Polytechnique de Paris',
       'Princeton University', 'University of Chicago',
       'Columbia University', 'Harvard University', 'Stanford University',
       'Harvard Business School',
       'Universidad Nacional Autonoma de Mexico', nan,
       'University of Arkansas', 'The University of Texas at Austin',
       'Trinity University', 'University of Central Arkansas',
       'Massachusetts Institute of Technology', 'University of Cambridge',
       'Nankai University', 'Stanford Graduate School of Business',
       'Oregon State University', 'The Hotchkiss School',
       'Bryn Mawr College', 'High School', 'Waseda University',
       'Shenzhen University', 'Hebrew University Jerusalem',
       'University of Electronic Science and Technology of China',
       'Pune University', 'SUNY Binghamton', 'New York University',
       'University of Wisconsin Madison', 'PSG College of Technology',
       'Technical University of Karlsruhe', 'B

In [35]:
## Wrong Value
# ArizonaCitizenshipUnited StatesMarital StatusMarriedChildren2, Berkeley), Santa Barbara

In [36]:
internal_data.loc[143,'College/University'] = np.nan

In [37]:
# Filling these index values with 'University of California'
internal_data.loc[[204, 251, 679, 705, 728, 764, 1327, 2349, 1596,197,1862,2542,2034, 61, 410,
                   663, 1666, 2182, 2300, 2322, 2340, 2587], 'College/University'] = 'University of California'

In [38]:
internal_data.loc[[1222,194,2024,81,610,1361,1845,1931,270,1527,1647,1813,2050,372,331,483,2092],'College/University'] = [
    'Anna University', 'University of Texas', 'Moscow Power Engineering Institute', 'University of Wisconsin', 'State Institute of Theatre Arts',
    'Moscow Institute of Radioengineering', 'University of Wisconsin', 'Stockholm University', 'Czech Technical University', 'University of North Carolina',
    'Rheinisch-Westfälische Technische Hochschule', 'Erasmus University Rotterdam', 'Hebrew University', 'Sir John Cass College',
    'University of Technology', 'University College London', 'Indian Institute of Technology (IIT) Kharagpur'
]

In [39]:
# Filling these index with 'Indian Institute of Technology'
internal_data.loc[[269, 1962],'College/University'] = 'Indian Institute of Technology'

In [40]:
internal_data.loc[[1082, 1253, 1396], 'College/University'] = 'Indian Institute of Management'

In [41]:
internal_data.loc[[1129, 1140], 'College/University'] = 'American University of Beirut'

In [42]:
internal_data.loc[2215, 'College/University'] = 'The University of North Carolina'

In [43]:
internal_data['College/University'].unique()

array(['University of Pennsylvania', 'Ecole Polytechnique de Paris',
       'Princeton University', 'University of Chicago',
       'Columbia University', 'Harvard University', 'Stanford University',
       'Harvard Business School',
       'Universidad Nacional Autonoma de Mexico', nan,
       'University of Arkansas', 'The University of Texas at Austin',
       'Trinity University', 'University of Central Arkansas',
       'Massachusetts Institute of Technology', 'University of Cambridge',
       'Nankai University', 'Stanford Graduate School of Business',
       'Oregon State University', 'The Hotchkiss School',
       'Bryn Mawr College', 'High School', 'Waseda University',
       'Shenzhen University', 'Hebrew University Jerusalem',
       'University of Electronic Science and Technology of China',
       'Pune University', 'SUNY Binghamton', 'New York University',
       'University of Wisconsin Madison', 'PSG College of Technology',
       'Technical University of Karlsruhe', 'U

In [44]:
internal_data.reset_index(inplace=True)
internal_data.drop(columns=['index'], inplace=True)

In [45]:
internal_data

,name,Wealth_History,personal_data,Citizenship,Marital Status,Education,College/University,Empire
0,Elon Musk,"[(2014, 8.4B), (2015, 12B), (2016, 10.7B), (20...","Age52Source of WealthTesla, SpaceX, Self MadeS...",United States,Single,Bachelor of Arts/Science,University of Pennsylvania,Self Made
1,Bernard Arnault,"[(2014, 33.5B), (2015, 37.2B), (2016, 34B), (2...","Age74Source of WealthLVMHResidenceParis, Franc...",France,Married,Bachelor of Arts/Science,Ecole Polytechnique de Paris,NaN
2,Jeff Bezos,"[(2014, 32B), (2015, 34.8B), (2016, 45.2B), (2...","Age59Source of WealthAmazon, Self MadeSelf-Mad...",United States,Engaged,Bachelor of Arts/Science,Princeton University,Self Made
3,Larry Ellison,"[(2014, 48B), (2015, 54.3B), (2016, 43.6B), (2...","Age78Source of WealthOracle, Self MadeSelf-Mad...",United States,In Relationship,Drop Out,University of Chicago,Self Made
4,Warren Buffett,"[(2014, 58.2B), (2015, 72.7B), (2016, 60.8B), ...","Age92Source of WealthBerkshire Hathaway, Self ...",United States,"Widowed, Remarried",Master of Science,Columbia University,Self Made
...,...,...,...,...,...,...,...,...
2600,Zhong Peifeng,"[(2021, 1.1B), (2022, 1.7B)]","Age60Source of Wealthsemiconductor, Self MadeR...",United States,Married,NaN,NaN,Self Made
2601,Kanye West,"[(2021, 1.8B), (2022, 2B)]","Age46Source of Wealthmusic, sneakers, Self Mad...",United States,Divorced,Drop Out,Chicago State University,Self Made
2602,Chung Yong-ji,NaN,"Age53Source of WealthBiotech, Self MadeResiden...",South Korea,NaN,Doctorate,Cornell University,Self Made
2603,Sam Bankman-Fried,"[(2021, 8.7B), (2022, 24B)]","Age31Source of Wealthcryptocurrency exchange, ...",United States,NaN,Bachelor of Arts/Science,Massachusetts Institute of Technology,Self Made


### Cleaning Wealth History Column

In [ ]:
# Creating Seperate Column by Year name that will have the worth value if available in wealth history column else NaN

In [47]:
def get_worth(year):
    l = []
    for i in internal_data['Wealth_History']:
        if type(i) == float:
            l.append({np.nan})
        if type(i) != float:
            s = {np.nan}
            for j in i:
                if year in j[0]:
                    s.add(j[1])
                else:
                    s.add(np.nan)
            l.append(s)   
        
    return l              

In [48]:
def fetch_worth(year):
    worth_value = []
    for i in internal_data[year]:
        found = False
        for element in i:
            if isinstance(element, str):
                worth_value.append(element)
                found = True
        if not found:
            worth_value.append(np.nan)
            
    return worth_value                       

#### Creating Year 2014

In [49]:
internal_data['2014'] = get_worth('2014')
internal_data['2014'] = fetch_worth('2014')

#### Creating Year 2015

In [50]:
internal_data['2015'] = get_worth('2015')
internal_data['2015'] = fetch_worth('2015')

#### Creating Year 2016

In [51]:
internal_data['2016'] = get_worth('2016')
internal_data['2016'] = fetch_worth('2016')

#### Creating Year 2017

In [52]:
internal_data['2017'] = get_worth('2017')
internal_data['2017'] = fetch_worth('2017')

#### Creating Year 2018

In [53]:
internal_data['2018'] = get_worth('2018')
internal_data['2018'] = fetch_worth('2018')

#### Creating Year 2019

In [54]:
internal_data['2019'] = get_worth('2019')
internal_data['2019'] = fetch_worth('2019')

#### Creating Year 2020

In [55]:
internal_data['2020'] = get_worth('2020')
internal_data['2020'] = fetch_worth('2020')

#### Creating Year 2021

In [56]:
internal_data['2021'] = get_worth('2021')
internal_data['2021'] = fetch_worth('2021')

#### Creating Year 2022

In [57]:
internal_data['2022'] = get_worth('2022')
internal_data['2022'] = fetch_worth('2022')

#### Creating Year 2023

In [58]:
internal_data['2023'] = get_worth('2023')
internal_data['2023'] = fetch_worth('2023')

In [59]:
internal_data

,name,Wealth_History,personal_data,Citizenship,Marital Status,Education,College/University,Empire,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Elon Musk,"[(2014, 8.4B), (2015, 12B), (2016, 10.7B), (20...","Age52Source of WealthTesla, SpaceX, Self MadeS...",United States,Single,Bachelor of Arts/Science,University of Pennsylvania,Self Made,8.4B,12B,10.7B,13.9B,19.9B,22.3B,24.6B,151B,219B,180B
1,Bernard Arnault,"[(2014, 33.5B), (2015, 37.2B), (2016, 34B), (2...","Age74Source of WealthLVMHResidenceParis, Franc...",France,Married,Bachelor of Arts/Science,Ecole Polytechnique de Paris,NaN,33.5B,37.2B,34B,41.5B,72B,76B,76B,150B,158B,211B
2,Jeff Bezos,"[(2014, 32B), (2015, 34.8B), (2016, 45.2B), (2...","Age59Source of WealthAmazon, Self MadeSelf-Mad...",United States,Engaged,Bachelor of Arts/Science,Princeton University,Self Made,32B,34.8B,45.2B,72.8B,112B,131B,113B,177B,171B,114B
3,Larry Ellison,"[(2014, 48B), (2015, 54.3B), (2016, 43.6B), (2...","Age78Source of WealthOracle, Self MadeSelf-Mad...",United States,In Relationship,Drop Out,University of Chicago,Self Made,48B,54.3B,43.6B,52.2B,58.5B,62.5B,59B,93B,106B,107B
4,Warren Buffett,"[(2014, 58.2B), (2015, 72.7B), (2016, 60.8B), ...","Age92Source of WealthBerkshire Hathaway, Self ...",United States,"Widowed, Remarried",Master of Science,Columbia University,Self Made,58.2B,72.7B,60.8B,75.6B,84B,82.5B,67.5B,96B,118B,106B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2600,Zhong Peifeng,"[(2021, 1.1B), (2022, 1.7B)]","Age60Source of Wealthsemiconductor, Self MadeR...",United States,Married,NaN,NaN,Self Made,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.1B,1.7B,NaN
2601,Kanye West,"[(2021, 1.8B), (2022, 2B)]","Age46Source of Wealthmusic, sneakers, Self Mad...",United States,Divorced,Drop Out,Chicago State University,Self Made,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.8B,2B,NaN
2602,Chung Yong-ji,NaN,"Age53Source of WealthBiotech, Self MadeResiden...",South Korea,NaN,Doctorate,Cornell University,Self Made,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2603,Sam Bankman-Fried,"[(2021, 8.7B), (2022, 24B)]","Age31Source of Wealthcryptocurrency exchange, ...",United States,NaN,Bachelor of Arts/Science,Massachusetts Institute of Technology,Self Made,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.7B,24B,NaN


In [60]:
internal_data.drop(columns=['Wealth_History', 'personal_data'], inplace=True)

In [61]:
# Convert 2015, and 2015 values in billion
internal_data[internal_data['2015'].str.contains('M', na=False)]

,name,Citizenship,Marital Status,Education,College/University,Empire,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
2398,John Lim,Singapore,Married,NaN,NaN,Self Made,565M,545M,520M,575M,780M,690M,645M,950M,1.1B,NaN
2498,Arash Ferdowsi,United States,Single,Drop Out,Massachusetts Institute of Technology,Self Made,NaN,500M,400M,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
internal_data.loc[2398,['2014','2015','2016','2017','2018','2019','2020','2021']] = ['0.56B','0.54B','0.52B','0.57B','0.78B','0.69B','0.64B','0.95B']

In [77]:
internal_data.loc[2498, ['2015','2016']] = ['0.5B','0.4B']

In [81]:
internal_data.iloc[[2398,2498]]

,name,Citizenship,Marital Status,Education,College/University,Empire,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
2398,John Lim,Singapore,Married,NaN,NaN,Self Made,0.56,0.54,0.52,0.57,0.78,0.69,0.64,0.95,1.1,NaN
2498,Arash Ferdowsi,United States,Single,Drop Out,Massachusetts Institute of Technology,Self Made,NaN,0.50,0.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Replacing 'B','' and converting type to float

In [79]:
internal_data['2014'] = internal_data['2014'].str.replace('B','').str.strip().astype(float)
internal_data['2015'] = internal_data['2015'].str.replace('B','').str.strip().astype(float)
internal_data['2016'] = internal_data['2016'].str.replace('B','').str.strip().astype(float)
internal_data['2017'] = internal_data['2017'].str.replace('B','').str.strip().astype(float)
internal_data['2018'] = internal_data['2018'].str.replace('B','').str.strip().astype(float)
internal_data['2019'] = internal_data['2019'].str.replace('B','').str.strip().astype(float)
internal_data['2020'] = internal_data['2020'].str.replace('B','').str.strip().astype(float)
internal_data['2021'] = internal_data['2021'].str.replace('B','').str.strip().astype(float)
internal_data['2022'] = internal_data['2022'].str.replace('B','').str.strip().astype(float)
internal_data['2023'] = internal_data['2023'].str.replace('B','').str.strip().astype(float)

In [80]:
internal_data

,name,Citizenship,Marital Status,Education,College/University,Empire,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Elon Musk,United States,Single,Bachelor of Arts/Science,University of Pennsylvania,Self Made,8.4,12.0,10.7,13.9,19.9,22.3,24.6,151.0,219.0,180.0
1,Bernard Arnault,France,Married,Bachelor of Arts/Science,Ecole Polytechnique de Paris,NaN,33.5,37.2,34.0,41.5,72.0,76.0,76.0,150.0,158.0,211.0
2,Jeff Bezos,United States,Engaged,Bachelor of Arts/Science,Princeton University,Self Made,32.0,34.8,45.2,72.8,112.0,131.0,113.0,177.0,171.0,114.0
3,Larry Ellison,United States,In Relationship,Drop Out,University of Chicago,Self Made,48.0,54.3,43.6,52.2,58.5,62.5,59.0,93.0,106.0,107.0
4,Warren Buffett,United States,"Widowed, Remarried",Master of Science,Columbia University,Self Made,58.2,72.7,60.8,75.6,84.0,82.5,67.5,96.0,118.0,106.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2600,Zhong Peifeng,United States,Married,NaN,NaN,Self Made,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.1,1.7,NaN
2601,Kanye West,United States,Divorced,Drop Out,Chicago State University,Self Made,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.8,2.0,NaN
2602,Chung Yong-ji,South Korea,NaN,Doctorate,Cornell University,Self Made,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2603,Sam Bankman-Fried,United States,NaN,Bachelor of Arts/Science,Massachusetts Institute of Technology,Self Made,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.7,24.0,NaN


In [ ]:
internal_data.to_csv('billionaires_personal_data.csv', encoding='utf-8')